In [1]:
import chess
from chess import Move
from copy import copy
import chess.svg
import tqdm
from tqdm import tqdm
import random
from random import choice
import tensorflow as tf
import numpy as np
import math
from abc import ABC, abstractmethod
from collections import defaultdict,deque
import math
from copy import copy


In [2]:
# keras model approach
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dense, Input,UpSampling2D
from tensorflow.keras.activations import relu

In [3]:
# ADDING THE CODE SO THAT TENSORFLOW DOES NOT EAT THE WHOLE GPU MEMORY
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
b = chess.Board()

#b.push_san("e4")
print(str(list(b.legal_moves)[0]))
print(type(str(list(b.legal_moves)[0])))
print(type(list(b.legal_moves)[0]))
print(dir(b.legal_moves)[0])

g1h3
<class 'str'>
<class 'chess.Move'>
__bool__


In [5]:
leg = list(b.legal_moves)
for i in leg:
    print(str(i))

g1h3
g1f3
b1c3
b1a3
h2h3
g2g3
f2f3
e2e3
d2d3
c2c3
b2b3
a2a3
h2h4
g2g4
f2f4
e2e4
d2d4
c2c4
b2b4
a2a4


In [6]:
'''import gym
import gym_chess
import random

env = gym.make('Chess-v0')
print(dir(env))
env.reset()
done = False

count = 0
while not done:
    print(env.render())
    print(env._board.fen())
    random.sample
    action = random.sample(env.legal_moves,1)
    a,b,done,d = env.step(action[0])
    state = copy(env)
    #count += 1
    #if count ==2:
    
env.close()'''

"import gym\nimport gym_chess\nimport random\n\nenv = gym.make('Chess-v0')\nprint(dir(env))\nenv.reset()\ndone = False\n\ncount = 0\nwhile not done:\n    print(env.render())\n    print(env._board.fen())\n    random.sample\n    action = random.sample(env.legal_moves,1)\n    a,b,done,d = env.step(action[0])\n    state = copy(env)\n    #count += 1\n    #if count ==2:\n    \nenv.close()"

In [7]:
def model_keras():
    
    input1 = Input(shape=(8,8,12))
    x = Conv2D(32,3, activation='relu',padding = "same",kernel_initializer="glorot_uniform")(input1)
    x = Conv2D(32,3, activation='relu',padding = "same",kernel_initializer="glorot_uniform")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=None, padding="same")(x)
    x = UpSampling2D()(x)
    
    x = Conv2D(64,3, activation='relu',padding = "same",kernel_initializer="glorot_uniform")(x)
    x = Conv2D(64,3, activation='relu',padding = "same",kernel_initializer="glorot_uniform")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=None, padding="same")(x)
    x = UpSampling2D()(x)
    
    x = Conv2D(128,5, activation='relu',padding = "same",kernel_initializer="glorot_uniform")(x)
    x = Conv2D(128,5, activation='relu',padding = "same",kernel_initializer="glorot_uniform")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=None, padding="same")(x)
    x = UpSampling2D()(x)
    
    x = Conv2D(256,5, activation='relu',padding = "same",kernel_initializer="glorot_uniform")(x)
    x = Conv2D(256,5, activation='relu',padding = "same",kernel_initializer="glorot_uniform")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=None, padding="same")(x)
    x = UpSampling2D()(x)
    
    x = Conv2D(256,5, activation='relu',padding = "same",kernel_initializer="glorot_uniform")(x)
    x = Conv2D(256,5, activation='relu',padding = "same",kernel_initializer="glorot_uniform")(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(256,5, activation='relu',padding = "same",kernel_initializer="glorot_uniform")(x)
    x = Conv2D(64,5, activation='sigmoid',padding = "same",kernel_initializer="glorot_uniform")(x)

    #here we introduce the second input (the encoded move)
    # 32 inputs 2*8 inputs from square to move from 
    # 2*8 from the square to move
    # exmple e2e5
    
    '''x = Flatten()(x)
    input2 = Input(shape=(32,))
    x = tf.keras.layers.Concatenate()([x, input2])
    
    x = Dense(1000,activation='relu',kernel_initializer="glorot_uniform")(x)
    x = BatchNormalization()(x)
    x = Dense(1000,activation='relu',kernel_initializer="glorot_uniform")(x)
    x = BatchNormalization()(x)
    x = Dense(1000,activation='relu',kernel_initializer="glorot_uniform")(x)
    x = BatchNormalization()(x)
    output = Dense(1,activation='sigmoid',kernel_initializer="glorot_uniform")(x)
    '''
    
    model = Model(inputs=input1, outputs=x, name="RL_Value_Function")
    
    print(model.summary())
    
    #model.compile(optimizer="adam",loss='binary_crossentropy',metrics=['binary_crossentropy'])
    
    return model
model_1 = model_keras()
model_2 = model_keras()

Model: "RL_Value_Function"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8, 8, 12)]        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 8, 8, 32)          3488      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 32)          9248      
_________________________________________________________________
batch_normalization (BatchNo (None, 8, 8, 32)          128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 4, 32)          0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)    

In [8]:
def encode(board):
    '''
    This is to encode a board position into the nn input
    we will have 12 8*8 boards
    first 6 will have the black pieces
    next 6 will have the white pieces
    
    todo::::
    
    we can later include some cues for castling and special moves
    
    
    
    The order on both of them will be 
    
    1 - pawn
    2 - rook
    3 - knights
    4 - bishop
    5 - queen
    6 - king
    
    The final encoding will have the board reversed (we will see it through blacks perspective)
    
    '''
    array = np.zeros((12, 8, 8), dtype=np.int) # these are the boards
    
    for square, piece in board.piece_map().items():
        x = int(square/8)
        y = int(square%8)

        # this is the black piece encoding
        
        if piece.symbol() == "p":
            array[0][x][y] = 1
            
        if piece.symbol() == "r":
            array[1][x][y] = 1
            
        if piece.symbol() == "n":
            array[2][x][y] = 1
            
        if piece.symbol() == "b":
            array[3][x][y] = 1
            
        if piece.symbol() == "q":
            array[4][x][y] = 1
        
        if piece.symbol() == "k":
            array[5][x][y] = 1
            
        # This is the white piece encoding
        
        if piece.symbol() == "P":
            array[6][x][y] = 1
            
        if piece.symbol() == "R":
            array[7][x][y] = 1
            
        if piece.symbol() == "N":
            array[8][x][y] = 1
            
        if piece.symbol() == "B":
            array[9][x][y] = 1
            
        if piece.symbol() == "Q":
            array[10][x][y] = 1
            
        if piece.symbol() == "K":
            array[11][x][y] = 1
    array.resize((8,8,12))
    array = tf.expand_dims(array,0)
    array = array.numpy()
    array = np.array(array,dtype = "float32")
    return array

temp_encode = encode(b)
def get_direction(x1,y1,x2,y2):
    x_diff = x2-x1
    y_diff = y2-y1
    direction = ""
    distance = None
    
    if x_diff==0 and y_diff>0:
        direction = 0 # North has a base of 0 (0-6 planes are of north)
        distance = abs(y_diff)
        
    elif x_diff==0 and y_diff<0:
        direction = 7 # South has a base of 7 (7-13 planes are of South)
        distance = abs(y_diff)
        
    elif x_diff>0 and y_diff==0:
        direction = 14 # East has a base of 14 (14-20 planes are of East)
        distance = abs(x_diff)
        
    elif x_diff<0 and y_diff==0:
        direction = 21 # West has a base of 21 (21-27 planes are of West)
        distance = abs(x_diff)
        
    elif x_diff>0 and y_diff>0:
        direction = 28 # NE has a base of 28 (28-34 planes are of NE)
        distance = abs(x_diff)
        
    elif x_diff<0 and y_diff>0: 
        direction = 35 # NW has a base of 35 (35-41 planes are of NW)
        distance = abs(x_diff)
        
    elif x_diff<0 and y_diff<0:
        direction = 42 # SW has a base of 42 (42-48 planes are of NW)
        distance = abs(x_diff)
        
    elif x_diff>0 and y_diff<0:
        direction = 49 # SE has a base of 49 (49-55 planes are of NW)
        distance = abs(x_diff)
    else:
        direction = "INVALID"
        distance = None
        
    return direction,distance

def decode_specific_move(move):
    '''Given a move it will find the specific position of the move in the NN'''
    board_mapping = {
        "a":0,
        "b":1,
        "c":2,
        "d":3,
        "e":4,
        "f":5,
        "g":6,
        "h":7,
        "1":0,
        "2":1,
        "3":2,
        "4":3,
        "5":4,
        "6":5,
        "7":6,
        "8":7,
    }
    
    temp_str = str(move)
    x1 = board_mapping[str(temp_str[0])]
    y1 = board_mapping[str(temp_str[1])]
    x2 = board_mapping[str(temp_str[2])]
    y2 = board_mapping[str(temp_str[3])]

    # ADD KNIGHT LOGIC FIRST
    
    knight_plane = knight_moves(x1,y1,x2,y2)
    if knight_plane !=0:
        return (x1,y1,knight_plane)
    else:
        # IF NOT KNIGHT MOVE THEN
        plane,dist = get_direction(x1,y1,x2,y2)
        final_plane = plane + dist
        return (x1,y1,final_plane)

def knight_moves(x1,y1,x2,y2):
    diff_x = x2-x1
    diff_y = y2-y1
    sq_x = math.pow((x2-x1),2)
    sq_y = math.pow((y2-y1),2)
    flag = 0
    plane = None
    if math.sqrt(sq_x+sq_y) == math.sqrt(5):
        flag = 1
    else:
        return 0
    if flag == 1 and diff_x<0 and diff_y == 1:
        plane = 56
    elif flag == 1 and diff_x<0 and diff_y == 2:
        plane = 57
    elif flag == 1 and diff_x>0 and diff_y == 2:
        plane = 58
    elif flag == 1 and diff_x>0 and diff_y == 1:
        plane = 59
    elif flag == 1 and diff_x>0 and diff_y == -1:
        plane = 60
    elif flag == 1 and diff_x>0 and diff_y == -2:
        plane = 61
    elif flag == 1 and diff_x<0 and diff_y == -2:
        plane = 62
    elif flag == 1 and diff_x<0 and diff_y == -1:
        plane = 63
    else:
        plane = 0
    return plane

def decode(nn_out,legal_moves):
    '''
    Currently the neural network will output 64 planes
    the first 56 planes will regard to all the queen moves
    Queen moves are 
    N,S,E,W,NE,NW,SE,SW
    
    for each direction of queen moves there will be number of steps that
    the piece will move
    (0,1,2,3,4,5,6,7)
    
    hence 56 planes for queen moves
    
    the next 8 planes are for knight moves,
    
    each of the knight plane will correspond to a Knight move
    
    '''
    move_values = {}
    board_mapping = {
        "a":0,
        "b":1,
        "c":2,
        "d":3,
        "e":4,
        "f":5,
        "g":6,
        "h":7,
        "1":0,
        "2":1,
        "3":2,
        "4":3,
        "5":4,
        "6":5,
        "7":6,
        "8":7,
    }
    for i in legal_moves:
        temp_str = str(i)
        x1 = board_mapping[str(temp_str[0])]
        y1 = board_mapping[str(temp_str[1])]
        x2 = board_mapping[str(temp_str[2])]
        y2 = board_mapping[str(temp_str[3])]
        
        # ADD KNIGHT LOGIC FIRST
        knight_plane = knight_moves(x1,y1,x2,y2)
        if knight_plane !=0:
            move_values[temp_str] = [nn_out[x1][y1][knight_plane],(x1,y1,knight_plane)]
        else:
            # IF NOT KNIGHT MOVE THEN
            plane,dist = get_direction(x1,y1,x2,y2)
            final_plane = plane + dist
            move_values[temp_str] = [nn_out[x1][y1][final_plane],(x1,y1,final_plane)]
    return move_values
def policy(move_dict,eps,turn):
    if np.random.rand() <= eps:  
        return random.choice(list(move_dict.keys()))
    # best move for white
    elif turn == True:
        action = max(move_dict,key = move_dict.get)
        return action
    # best move for black
    else:
        action = min(move_dict,key = move_dict.get)
        return action

In [9]:
temp_out = model_1.predict(temp_encode)

In [10]:
decode_specific_move("e2e4")

(4, 1, 2)

In [11]:
temp_encode.shape

(1, 8, 8, 12)

In [12]:
leg = list(b.legal_moves)
decoded = decode(temp_out[0],leg)
print(max(decoded,key = decoded.get))
print(decoded)

g1f3
{'g1h3': [0.5001886, (6, 0, 58)], 'g1f3': [0.5008512, (6, 0, 57)], 'b1c3': [0.5005626, (1, 0, 58)], 'b1a3': [0.5003201, (1, 0, 57)], 'h2h3': [0.5000309, (7, 1, 1)], 'g2g3': [0.5001399, (6, 1, 1)], 'f2f3': [0.5000927, (5, 1, 1)], 'e2e3': [0.5006475, (4, 1, 1)], 'd2d3': [0.50002503, (3, 1, 1)], 'c2c3': [0.49992576, (2, 1, 1)], 'b2b3': [0.49996966, (1, 1, 1)], 'a2a3': [0.49971575, (0, 1, 1)], 'h2h4': [0.5003801, (7, 1, 2)], 'g2g4': [0.5004841, (6, 1, 2)], 'f2f4': [0.49991295, (5, 1, 2)], 'e2e4': [0.49979472, (4, 1, 2)], 'd2d4': [0.49992776, (3, 1, 2)], 'c2c4': [0.49966356, (2, 1, 2)], 'b2b4': [0.49942294, (1, 1, 2)], 'a2a4': [0.49952796, (0, 1, 2)]}


In [13]:
random.choice(list(decoded.keys()))

'c2c3'

In [14]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-4)

In [15]:
#@tf.function
def create_y_true(current_q,current_r,lamb):
    return tf.math.multiply(current_q,lamb) + current_r
#@tf.function
def custom_loss(y_true,y_pred):
    return tf.keras.losses.MSE(y_true,y_pred)
def update_target_network():
    model_2.set_weights(model_1.get_weights())

In [16]:
#@tf.function

def train_step_keras(model_1,model_2,gamma,batch_size,epsilon):
    #decaying the exploration
    if epsilon['epsilon'] > epsilon['epsilon_min']:
         epsilon['epsilon'] =  epsilon['epsilon'] * epsilon['epsilon_decay']
    
    batch = random.sample(replay,batch_size)
    batch_reward = []
    batch_action = []
    batch_done = []
    batch_next_legal = []
    
    batch_current_state = np.zeros((batch_size, 8, 8, 12))
    batch_next_state = np.zeros((batch_size, 8, 8, 12))
    
    for i in range(batch_size):
        batch_reward.append(batch[i][2])
        batch_action.append(batch[i][1])
        batch_current_state[i] = batch[i][0]
        batch_next_state[i] = batch[i][3]
        batch_next_legal.append(batch[i][4])
        
     #lets calculate the next state value as the current value will be calculated in 
    # in gradient tape
    # also we need to form the target
    
    next_q = model_2.predict(batch_next_state)

    target = []
    for c,i in enumerate(next_q):
        move_dict = decode(i,curr_legal_moves)
        action = max(move_dict,key = move_dict.get)
        temp = move_dict[action][0]*gamma + batch_reward[c]
        target.append(temp)
        
    with tf.GradientTape() as tape:
        # logits is the forward pass
        logits = model_1(batch_current_state, training=True)
        
        q_target = np.array(logits)
        
        # VERY IMPORTANT NOTE, IF THE EPISODE ENDS THE DONE VALUE BECOMES TRUE
        # IT IS VERY IMPORTANT THAT THE NN UPDATES TOWARDS THIS TRUE VALUE RATHER THAN
        # ITS OWN THINKING VALUE (r + gamma*max(action)) THAT WE USE FOR ALL
        # NON TERMINAL REWARDS 
        # THIS MAKES OR BREAKS THE NETWORK VERY VERY IMPORTANT
        for i in range(batch_size):
            nn_move_location = decode_specific_move(batch_action[i])
            q_target[i][nn_move_location] = target[i]
        
        # calculating the loss
        loss_value = custom_loss(q_target,logits)
    
    #we retrieve the gradients
    grads = tape.gradient(loss_value, model_1.trainable_weights)
    
    #THIS IS ONE STEP OF GRAD DESCENT (Minimizes the loss)
    optimizer.apply_gradients(zip(grads, model_1.trainable_weights))


In [17]:
b.turn

True

In [18]:
#DEFINING THE CONSTANTS
moves = 0
# lets build some memory into the model to perform decorrelated batch updates
# this is TD learning

# so apparantly the NN has to be adjusted only according to the action taken by it
# for example if action 2 is taken then only the weights for action 2 should be changed
# for this reason we should only update the q_2 vector with regarding the chosen action
# rewards will also be added to that action only
# q_2 is what we thought the value of the state will be after doing action
# we also add the reward and make this the target for the NN

replay = deque(maxlen = 3000)
gamma = 0.99
warmup = 5 #will start training after these many episodes have passed
batch_size = 64
# to balance exploration
epsilon = {
"epsilon" : 1.0,
"epsilon_decay": 0.999,
"epsilon_min":0.01,
}

In [22]:
# SELF PLAY
#%%time
#%%svg

def calculate_reward(board):
    if board.result() == "*":
        return 0
    elif board.result() == "0-1":
        return -1
    elif board.result() == "1-0":
        return 1
    elif board.result() == "1/2-1/2":
        return 0.5
update_target_network()
    
    
for i in tqdm(range(100)):
    board = chess.Board()
    done = False
    while not done:
        current_encoded_state = encode(board)
        
        # this is the current prediction for the current situation
        nn_out = model_1.predict(current_encoded_state)
        curr_legal_moves = list(board.legal_moves)
        
        decoded_moves = decode(nn_out[0],curr_legal_moves)
        
        #Epsilon greedy policy
        action = policy(decoded_moves,epsilon['epsilon'],board.turn)

        board.push(Move.from_uci(action))
        # calculating reward from the current board position
        reward = calculate_reward(board)
        
        #this is the new state
        new_state_encoded = encode(board)
        new_state_legal_moves = list(board.legal_moves)
        # we can only record or make action replay for all white moves
        # or moves that we want to maximize on as black automatically gets better
        # when white gets better
        
        if board.turn == True:
            replay.append((current_encoded_state,action,reward,new_state_encoded,new_state_legal_moves,i))
            
        if i>warmup:
            train_step_keras(model_1,model_2,gamma,batch_size,epsilon)
        
        moves = moves + 1
        
        if board.result() != "*":
            print(board.result())
            done = True
            update_target_network()

  1%|          | 1/100 [00:00<01:21,  1.22it/s]

1/2-1/2


  2%|▏         | 2/100 [00:01<01:20,  1.22it/s]

1/2-1/2


  3%|▎         | 3/100 [00:02<01:17,  1.25it/s]

...

1/2-1/2


 99%|█████████▉| 99/100 [22:28<00:12, 12.23s/it]

1/2-1/2


100%|██████████| 100/100 [22:48<00:00, 13.68s/it]

1/2-1/2


In [21]:
# TESTING PART NO LEARNING

for i in tqdm(range(100)):
    board = chess.Board()
    done = False
    while not done:
        current_encoded_state = encode(board)
        
        # this is the current prediction for the current situation
        nn_out = model_1.predict(current_encoded_state)
        curr_legal_moves = list(board.legal_moves)
        
        decoded_moves = decode(nn_out[0],curr_legal_moves)
        print(decoded_moves)
        #Epsilon greedy policy
        action = policy(decoded_moves,epsilon['epsilon'],board.turn)

        board.push(Move.from_uci(action))
        # calculating reward from the current board position
        reward = calculate_reward(board)
        
        #this is the new state
        new_state_encoded = encode(board)
        new_state_legal_moves = list(board.legal_moves)
        # we can only record or make action replay for all white moves
        # or moves that we want to maximize on as black automatically gets better
        # when white gets better
        moves = moves + 1
        
        if board.result() != "*":
            print(board.result())
            done = True

  0%|          | 0/100 [00:00<?, ?it/s]

{'g1h3': [1.0, (6, 0, 58)], 'g1f3': [1.0, (6, 0, 57)], 'b1c3': [1.0, (1, 0, 58)], 'b1a3': [1.0, (1, 0, 57)], 'h2h3': [1.0, (7, 1, 1)], 'g2g3': [1.0, (6, 1, 1)], 'f2f3': [1.0, (5, 1, 1)], 'e2e3': [1.0, (4, 1, 1)], 'd2d3': [1.0, (3, 1, 1)], 'c2c3': [1.0, (2, 1, 1)], 'b2b3': [1.0, (1, 1, 1)], 'a2a3': [1.0, (0, 1, 1)], 'h2h4': [1.0, (7, 1, 2)], 'g2g4': [1.0, (6, 1, 2)], 'f2f4': [1.0, (5, 1, 2)], 'e2e4': [1.0, (4, 1, 2)], 'd2d4': [1.0, (3, 1, 2)], 'c2c4': [1.0, (2, 1, 2)], 'b2b4': [1.0, (1, 1, 2)], 'a2a4': [1.0, (0, 1, 2)]}
{'g8h6': [1.0, (6, 7, 61)], 'g8f6': [1.0, (6, 7, 62)], 'b8c6': [1.0, (1, 7, 61)], 'b8a6': [1.0, (1, 7, 62)], 'h7h6': [1.0, (7, 6, 8)], 'g7g6': [1.0, (6, 6, 8)], 'f7f6': [1.0, (5, 6, 8)], 'e7e6': [1.0, (4, 6, 8)], 'd7d6': [1.0, (3, 6, 8)], 'c7c6': [1.0, (2, 6, 8)], 'b7b6': [1.0, (1, 6, 8)], 'a7a6': [1.0, (0, 6, 8)], 'h7h5': [1.0, (7, 6, 9)], 'g7g5': [1.0, (6, 6, 9)], 'f7f5': [1.0, (5, 6, 9)], 'e7e5': [1.0, (4, 6, 9)], 'd7d5': [1.0, (3, 6, 9)], 'c7c5': [1.0, (2, 6, 9)], 'b

{'h8g7': [0.9916089, (7, 7, 43)], 'e8f7': [0.9891088, (4, 7, 50)], 'b7b8': [0.9894211, (1, 6, 1)], 'b7a7': [0.99934065, (1, 6, 22)], 'f6g8': [0.9996564, (5, 5, 58)], 'f6h7': [0.9883551, (5, 5, 59)], 'f6h5': [0.98873997, (5, 5, 60)], 'f6d5': [0.98972756, (5, 5, 63)], 'f6g4': [0.9884565, (5, 5, 61)], 'f6e4': [0.9896481, (5, 5, 62)], 'a6b8': [0.98906195, (0, 5, 58)], 'a6c5': [0.9889665, (0, 5, 60)], 'a6b4': [0.98895204, (0, 5, 61)], 'b2c1q': [0.98955864, (1, 1, 50)], 'b2c1r': [0.98955864, (1, 1, 50)], 'b2c1b': [0.98955864, (1, 1, 50)], 'b2c1n': [0.98955864, (1, 1, 50)], 'b2a1q': [0.9887942, (1, 1, 43)], 'b2a1r': [0.9887942, (1, 1, 43)], 'b2a1b': [0.9887942, (1, 1, 43)], 'b2a1n': [0.9887942, (1, 1, 43)], 'e7e6': [0.98964, (4, 6, 8)], 'd7d6': [0.99307346, (3, 6, 8)], 'c7c6': [0.99016654, (2, 6, 8)], 'b6b5': [0.99082106, (1, 5, 8)], 'e7e5': [0.9896637, (4, 6, 9)], 'd7d5': [0.98880976, (3, 6, 9)], 'c7c5': [0.9886097, (2, 6, 9)]}
{'f7f8': [0.9900951, (5, 6, 1)], 'f7h7': [0.9896116, (5, 6, 16)]

{'h8g7': [0.99208593, (7, 7, 43)], 'b7b8': [0.98998606, (1, 6, 1)], 'b7a7': [0.99932563, (1, 6, 22)], 'g6h7': [0.98948604, (6, 5, 29)], 'g6g7': [0.99030054, (6, 5, 1)], 'g6f7': [0.99449, (6, 5, 36)], 'g6h6': [0.99047816, (6, 5, 15)], 'g6g5': [0.99405754, (6, 5, 8)], 'g6f5': [0.9908246, (6, 5, 43)], 'd7d6': [0.9951587, (3, 6, 8)], 'c7c6': [0.9926058, (2, 6, 8)], 'f6f5': [0.99591494, (5, 5, 8)], 'b6b5': [0.9927007, (1, 5, 8)], 'd7d5': [0.9903681, (3, 6, 9)], 'c7c5': [0.99003375, (2, 6, 9)]}
{'d4d7': [0.9899602, (3, 3, 3)], 'd4f6': [0.99318665, (3, 3, 30)], 'd4d6': [0.9928372, (3, 3, 2)], 'd4b6': [0.99072385, (3, 3, 37)], 'd4e5': [0.9934072, (3, 3, 29)], 'd4d5': [0.9981625, (3, 3, 1)], 'd4c5': [0.9892481, (3, 3, 36)], 'd4h4': [0.9919764, (3, 3, 18)], 'd4g4': [0.99077153, (3, 3, 17)], 'd4f4': [0.99194574, (3, 3, 16)], 'd4e4': [0.9980556, (3, 3, 15)], 'd4c4': [0.9919025, (3, 3, 22)], 'd4b4': [0.99164987, (3, 3, 23)], 'd4a4': [0.9914735, (3, 3, 24)], 'd4c3': [0.99051493, (3, 3, 43)], 'f3b7':

{'h8g7': [0.99205625, (7, 7, 43)], 'h7g8': [0.9897087, (7, 6, 36)], 'h7g7': [0.9880219, (7, 6, 22)], 'h7h6': [0.99448967, (7, 6, 8)], 'h7g6': [0.9893711, (7, 6, 43)], 'b7b8': [0.98996043, (1, 6, 1)], 'b7a7': [0.99933505, (1, 6, 22)], 'd7d6': [0.994878, (3, 6, 8)], 'c7c6': [0.99227977, (2, 6, 8)], 'f6f5': [0.9956602, (5, 5, 8)], 'b6b5': [0.99242353, (1, 5, 8)], 'd7d5': [0.9901771, (3, 6, 9)], 'c7c5': [0.9898416, (2, 6, 9)]}
{'d4d7': [0.98958665, (3, 3, 3)], 'd4f6': [0.99300086, (3, 3, 30)], 'd4d6': [0.9924568, (3, 3, 2)], 'd4b6': [0.99059975, (3, 3, 37)], 'd4e5': [0.9930905, (3, 3, 29)], 'd4d5': [0.9980962, (3, 3, 1)], 'd4c5': [0.98890173, (3, 3, 36)], 'd4h4': [0.9917346, (3, 3, 18)], 'd4g4': [0.99048686, (3, 3, 17)], 'd4f4': [0.99164855, (3, 3, 16)], 'd4e4': [0.99796534, (3, 3, 15)], 'd4c4': [0.99162817, (3, 3, 22)], 'd4b4': [0.9914396, (3, 3, 23)], 'd4a4': [0.9913517, (3, 3, 24)], 'd4c3': [0.99024224, (3, 3, 43)], 'g3h4': [0.9953001, (6, 2, 29)], 'g3g4': [0.9910917, (6, 2, 1)], 'g3f4'

...


{'f5h7': [0.9962872, (5, 4, 30)], 'f5d7': [0.9910693, (5, 4, 37)], 'f5g6': [0.98917437, (5, 4, 29)], 'f5f6': [0.99935365, (5, 4, 1)], 'f5e6': [0.9908884, (5, 4, 36)], 'f5h5': [0.9922197, (5, 4, 16)], 'f5g5': [0.99800056, (5, 4, 15)], 'f5e5': [0.9891749, (5, 4, 22)], 'f5d5': [0.98854244, (5, 4, 23)], 'f5c5': [0.99169433, (5, 4, 24)], 'f5g4': [0.99198234, (5, 4, 50)], 'f5f4': [0.99982953, (5, 4, 8)], 'f5e4': [0.9889451, (5, 4, 43)], 'f5d3': [0.9891167, (5, 4, 44)], 'f5c2': [0.9920312, (5, 4, 45)], 'f3b7': [0.98878306, (5, 2, 39)], 'f3c6': [0.9896696, (5, 2, 38)], 'f3h5': [0.9931439, (5, 2, 30)], 'f3d5': [0.99062353, (5, 2, 37)], 'f3g4': [0.9930167, (5, 2, 29)], 'f3e4': [0.9929912, (5, 2, 36)], 'f3g2': [0.9924218, (5, 2, 50)], 'f3h1': [0.9909425, (5, 2, 51)], 'h2g3': [0.9981785, (7, 1, 36)], 'h2g2': [0.9905592, (7, 1, 22)], 'h2h1': [0.99747795, (7, 1, 8)], 'b2f6': [0.9891888, (1, 1, 32)], 'b2e5': [0.9902708, (1, 1, 31)], 'b2d4': [0.98819184, (1, 1, 30)], 'b2c3': [0.9970468, (1, 1, 29)], 

{'g8h6': [1.0, (6, 7, 61)], 'g8f6': [1.0, (6, 7, 62)], 'b8c6': [1.0, (1, 7, 61)], 'b8a6': [1.0, (1, 7, 62)], 'a8a7': [1.0, (0, 7, 8)], 'a8a6': [1.0, (0, 7, 9)], 'g7h6': [1.0, (6, 6, 50)], 'g7g6': [1.0, (6, 6, 8)], 'f7f6': [1.0, (5, 6, 8)], 'e7e6': [1.0, (4, 6, 8)], 'd7d6': [1.0, (3, 6, 8)], 'c7c6': [1.0, (2, 6, 8)], 'b7b6': [1.0, (1, 6, 8)], 'a5a4': [1.0, (0, 4, 8)], 'g7g5': [1.0, (6, 6, 9)], 'f7f5': [1.0, (5, 6, 9)], 'e7e5': [1.0, (4, 6, 9)], 'd7d5': [1.0, (3, 6, 9)], 'c7c5': [1.0, (2, 6, 9)], 'b7b5': [1.0, (1, 6, 9)]}
{'h1h5': [1.0, (7, 0, 4)], 'h1h4': [1.0, (7, 0, 3)], 'h1h3': [1.0, (7, 0, 2)], 'h1h2': [1.0, (7, 0, 1)], 'g1h3': [1.0, (6, 0, 58)], 'g1f3': [1.0, (6, 0, 57)], 'b1c3': [1.0, (1, 0, 58)], 'b1a3': [1.0, (1, 0, 57)], 'h6g7': [1.0, (7, 5, 36)], 'g2g3': [1.0, (6, 1, 1)], 'f2f3': [1.0, (5, 1, 1)], 'e2e3': [1.0, (4, 1, 1)], 'd2d3': [1.0, (3, 1, 1)], 'c2c3': [1.0, (2, 1, 1)], 'b2b3': [1.0, (1, 1, 1)], 'a2a3': [1.0, (0, 1, 1)], 'g2g4': [1.0, (6, 1, 2)], 'f2f4': [1.0, (5, 1, 2)], 

  2%|▏         | 2/100 [00:01<01:27,  1.12it/s]


{'f7f8': [0.99055076, (5, 6, 1)], 'f7h7': [0.9897164, (5, 6, 16)], 'f7g7': [0.98983884, (5, 6, 15)], 'f7e7': [0.99241674, (5, 6, 22)], 'g3h4': [0.99359876, (6, 2, 29)], 'g3g4': [0.9892427, (6, 2, 1)], 'g3h3': [0.9969828, (6, 2, 15)], 'g3f3': [0.9890462, (6, 2, 22)], 'g3h2': [0.9899992, (6, 2, 50)], 'b2e5': [0.9896853, (1, 1, 31)], 'b2d4': [0.98744404, (1, 1, 30)], 'b2c3': [0.9966296, (1, 1, 29)], 'b2a3': [0.9919721, (1, 1, 36)], 'b2c1': [0.98972964, (1, 1, 50)], 'g1h3': [0.9896999, (6, 0, 58)], 'g1f3': [0.993484, (6, 0, 57)], 'd1e1': [0.9957881, (3, 0, 15)], 'd1c1': [0.9902278, (3, 0, 22)], 'b1c3': [0.9906666, (1, 0, 58)], 'b1a3': [0.99009454, (1, 0, 57)], 'f6e7': [0.9921547, (5, 5, 36)], 'g2h3': [0.9998926, (6, 1, 29)], 'e2e3': [0.9968322, (4, 1, 1)], 'd2d3': [0.9950526, (3, 1, 1)], 'c2c3': [0.99143577, (2, 1, 1)], 'a2a3': [0.9916134, (0, 1, 1)], 'e2e4': [0.99659187, (4, 1, 2)], 'd2d4': [0.993503, (3, 1, 2)], 'c2c4': [0.9917782, (2, 1, 2)], 'a2a4': [0.9890563, (0, 1, 2)]}
{'h8g7': [0

KeyboardInterrupt: 